In [4]:
import tensorflow as tf

In [5]:
# A shape is (N, P_A, C), B shape is (N, P_B, C)
# D shape is (N, P_A, P_B)                                                             
def batch_distance_matrix_general(A, B):
    """L2 squared"""
    r_A = tf.reduce_sum(A * A, axis=2, keepdims=True)                                  
    r_B = tf.reduce_sum(B * B, axis=2, keepdims=True)
    m = tf.matmul(A, tf.transpose(B, perm=(0, 2, 1)))
    D = r_A - 2 * m + tf.transpose(r_B, perm=(0, 2, 1))
    return D  

In [6]:
A = tf.constant([[[1.0, 2.0, 3.0], [2.0, 3.0, 4.0]]])
B = tf.constant([[[1.1, 3.1, 4.1], [1.2, 3.2, 4.2], [1.3, 3.3, 4.3]]])
with tf.Session() as sess:
    print(batch_distance_matrix_general(A, B).eval())

[[[2.4300022  2.9199963  3.4700012 ]
  [0.83000374 0.7199993  0.66999817]]]


In [7]:
# A shape is (N, P_A, C), B shape is (N, P_B, C)
# D shape is (N, P_A, P_B)                                                             
def batch_distance_matrix_general_l1_(A, B):
    """L1 norm"""
    Ac = tf.transpose(tf.expand_dims(A, axis=-1), perm=(0,2,1,3))
    Bc = tf.transpose(tf.expand_dims(B, axis=-1), perm=(0,2,1,3))
    #A_1s = tf.expand_dims(tf.ones_like(Ac), axis=3)
    #B_1s = tf.expand_dims(tf.ones_like(Bc), axis=2)
    A_1s = tf.ones_like(Ac)
    B_1s = tf.ones_like(Bc)
    
    print(A_1s.shape)
    print(A_1s.eval())
    print(B_1s.shape)
    print(B_1s.eval())
    
    A_exp = tf.matmul(Ac, tf.transpose(B_1s, perm=(0,1,3,2)))
    print("A_exp", A_exp.shape)
    print(A_exp.eval())
    B_exp = tf.matmul(A_1s, tf.transpose(Bc, perm=(0,1,3,2)))
    print("B_exp", B_exp.shape)
    print(B_exp.eval())
    
    D = tf.reduce_max(tf.abs(A_exp - B_exp), axis=1)
    print("diff", D.shape)
    print(D.eval())
    
    return D
with tf.Session() as sess:
    print(batch_distance_matrix_general_l1_(A, B).eval())

(1, 3, 2, 1)
[[[[1.]
   [1.]]

  [[1.]
   [1.]]

  [[1.]
   [1.]]]]
(1, 3, 3, 1)
[[[[1.]
   [1.]
   [1.]]

  [[1.]
   [1.]
   [1.]]

  [[1.]
   [1.]
   [1.]]]]
A_exp (1, 3, 2, 3)
[[[[1. 1. 1.]
   [2. 2. 2.]]

  [[2. 2. 2.]
   [3. 3. 3.]]

  [[3. 3. 3.]
   [4. 4. 4.]]]]
B_exp (1, 3, 2, 3)
[[[[1.1 1.2 1.3]
   [1.1 1.2 1.3]]

  [[3.1 3.2 3.3]
   [3.1 3.2 3.3]]

  [[4.1 4.2 4.3]
   [4.1 4.2 4.3]]]]
diff (1, 2, 3)
[[[1.0999999  1.2        1.3000002 ]
  [0.9        0.79999995 0.70000005]]]
[[[1.0999999  1.2        1.3000002 ]
  [0.9        0.79999995 0.70000005]]]


In [33]:
# A shape is (N, P_A, C), B shape is (N, P_B, C)
# D shape is (N, P_A, P_B)                                                             
def batch_distance_matrix_general_l1(A, B):
    """L1 norm"""
    # Add one dimension
    Ac = tf.transpose(tf.expand_dims(A, axis=-1), perm=(0,2,1,3))  # (N, C, P_A, 1)
    Bc = tf.transpose(tf.expand_dims(B, axis=-1), perm=(0,2,3,1))  # (N, C, 1, P_B)
    A_1s = tf.ones_like(Ac)
    B_1s = tf.ones_like(Bc)
    
    A_exp = tf.matmul(Ac, B_1s)  # (N, C, P_A, P_B) where each A[ijk] value is copied along x in A_exp[ijkx]
    B_exp = tf.matmul(A_1s, Bc)  # (N, C, P_A, P_B) where each B[ijk] value is copied along x in B_exp[ijxk]
    # A_exp - B_exp is now pairwise a_i - b_j for each channel :-)
    
    D = tf.reduce_max(tf.abs(A_exp - B_exp), axis=1)
    print(D.shape)
    return D
with tf.Session() as sess:
    print(batch_distance_matrix_general_l1(A, B).eval())

(1, 2, 3)
[[[1.0999999  1.2        1.3000002 ]
  [0.9        0.79999995 0.70000005]]]


In [32]:
import math
import numpy as np
import scipy as sp
import scipy.special
C = 3
print("C=2 --> ", np.sqrt(3.141529))
print("C=3 --> ", np.power(3.141529 * 4/3, 1/3))
np.sqrt(3.141592) / np.exp(1.0 / C * sp.special.loggamma(0.5 * C + 1))

C=2 -->  1.7724358944684007
C=3 -->  1.611981066781987


1.6119917863337592